In [ ]:
## read Waltz database ##
import pandas as pd
data = pd.read_csv('waltzdb_export.csv').drop(497).reset_index(drop=True)
#print(data)

In [ ]:

## split sequences into amino acids ##
seq = data['Sequence']
s = [] # list of amino acids of each hexapeptide
for i in range(0,len(seq)):
    s.append(list(seq[i]))
#print(s)



In [ ]:

## count similarity ##
del_index = [] # a list of example index need to be deleted

for i in range(0,len(data)):
    
    for j in range(i+1,len(data)):
        similarity = 0.0 # initialize similarity
        
        for k in range(0,6):
            if s[i][k]==s[j][k]:
                similarity = similarity + 1.0
                
        similarity = similarity/6.0
        
        if similarity>=0.5:
            del_index.append(j)
            
del_index = list(set(del_index))        

In [ ]:

## delete hexapeptides with similarity>=50% ##
data = data.drop(del_index).reset_index(drop=True)

## split sequences again with hexapeptides remained ##
seq = data['Sequence']
s = [] # list of amino acids of each hexapeptide
for i in range(0,len(seq)):
    s.append(list(seq[i]))

In [ ]:

## split sequences into columns ##
data['1st_aa'] = [ s[i][0] for i in range(0,len(data))]
data['2nd_aa'] = [ s[i][1] for i in range(0,len(data))]
data['3rd_aa'] = [ s[i][2] for i in range(0,len(data))]
data['4th_aa'] = [ s[i][3] for i in range(0,len(data))]
data['5th_aa'] = [ s[i][4] for i in range(0,len(data))]
data['6th_aa'] = [ s[i][5] for i in range(0,len(data))]



In [ ]:

## read aaindex with re ##

import re

title = []
f = open('aaindex1')
#try:
string =f.readlines()
#finally:
f.close()

for i in range(0,len(string)):
    title.append(''.join(re.findall("^H\s+(.+)",string[i])))
    
while '' in title:
    title.remove('')
    
#print(len(title))    
#print(title)
#for line in title:
#    print(line)
#type(title[0])

properties = []
for i in range(0,len(string)):
    if string[i][0] == 'I':
        properties.append(re.findall("(NA|-*\d*\.?\d*)",string[i+1]))
        properties.append(re.findall("(NA|-*\d*\.?\d*)",string[i+2]))
        #aa.append(re.findall("^I\s+(.+)",str[i]))

for i in range(0,len(properties)):
    while '' in properties[i]:
        properties[i].remove('')



In [ ]:

## put aaindex into dataframe and delete NA ##

import pandas as pd
aaindex = pd.DataFrame(columns = title)
#print(aaindex)
test_aaindex = dict()

for i in range(0,len(title)):
    properties[2*i].extend(properties[2*i+1])
    aaindex.iloc[:,i] = properties[2*i]
    #test_aaindex['A'][i] = properties[2*i]
    
for i in range(0,len(title)):
    for j in range(0,20):
        if aaindex.iloc[j][title[i]] == "NA":
            aaindex = aaindex.drop(title[i],axis=1)
            break

            
#aaindex.reindex(index=['A'])
#print(aaindex)

In [ ]:

## set aaindex into dictionary for faster retrieval ##

aaindex_dict = list()

for i in range(0, len(aaindex.columns)):
    aaindex_dict.append(dict())
    # 20 amino acid properties.
    aaindex_dict[i]['A'] = float(aaindex.iloc[0][i])
    aaindex_dict[i]['R'] = float(aaindex.iloc[1][i])
    aaindex_dict[i]['N'] = float(aaindex.iloc[2][i])
    aaindex_dict[i]['D'] = float(aaindex.iloc[3][i])
    aaindex_dict[i]['C'] = float(aaindex.iloc[4][i])
    aaindex_dict[i]['Q'] = float(aaindex.iloc[5][i])
    aaindex_dict[i]['E'] = float(aaindex.iloc[6][i])
    aaindex_dict[i]['G'] = float(aaindex.iloc[7][i])
    aaindex_dict[i]['H'] = float(aaindex.iloc[8][i])
    aaindex_dict[i]['I'] = float(aaindex.iloc[9][i])
    aaindex_dict[i]['L'] = float(aaindex.iloc[10][i])
    aaindex_dict[i]['K'] = float(aaindex.iloc[11][i])
    aaindex_dict[i]['M'] = float(aaindex.iloc[12][i])
    aaindex_dict[i]['F'] = float(aaindex.iloc[13][i])
    aaindex_dict[i]['P'] = float(aaindex.iloc[14][i])
    aaindex_dict[i]['S'] = float(aaindex.iloc[15][i])
    aaindex_dict[i]['T'] = float(aaindex.iloc[16][i])
    aaindex_dict[i]['W'] = float(aaindex.iloc[17][i])
    aaindex_dict[i]['Y'] = float(aaindex.iloc[18][i])
    aaindex_dict[i]['V'] = float(aaindex.iloc[19][i])

#print(aaindex_dict)
#len(aaindex_dict)

In [ ]:

## feature engineering ##

# basic sequence features#

first_aa = pd.get_dummies(data['1st_aa'], prefix = '1st_aa')#, drop_first=True)
second_aa = pd.get_dummies(data['2nd_aa'], prefix = '2nd_aa')#, drop_first=True)
third_aa = pd.get_dummies(data['3rd_aa'], prefix = '3rd_aa')#, drop_first=True)
fourth_aa = pd.get_dummies(data['4th_aa'], prefix = '4th_aa')#, drop_first=True)
fifth_aa = pd.get_dummies(data['5th_aa'], prefix = '5th_aa')#, drop_first=True)
sixth_aa = pd.get_dummies(data['6th_aa'], prefix = '6th_aa')#, drop_first=True)

y = data['Classification']
x = first_aa.join(second_aa).join(third_aa).join(fourth_aa).join(fifth_aa).join(sixth_aa)


print(x)




In [ ]:

# all physicochemical features #
for i in range(0,len(aaindex.columns)):
    #prop = []

    for k in ['1st_aa','2nd_aa','3rd_aa','4th_aa','5th_aa','6th_aa']:
        
        prop = [] # a list of physicochemical property values
        
        for j in range(0,len(data)):
            
            prop.append(aaindex_dict[i][data.iloc[j][k]])
                
        x['%s_%s' %(k,aaindex.columns[i])] = prop

In [ ]:

# feature selection #

feat = ['1st_aa_NOZY710101','2nd_aa_NOZY710101','3rd_aa_NOZY710101','4th_aa_NOZY710101','5th_aa_NOZY710101',
      '6th_aa_NOZY710101',
      '1st_aa_VASM830103','2nd_aa_VASM830103','3rd_aa_VASM830103','4th_aa_VASM830103','5th_aa_VASM830103',
      '6th_aa_VASM830103',
       '1st_aa_PALJ810104','2nd_aa_PALJ810104','3rd_aa_PALJ810104','4th_aa_PALJ810104','5th_aa_PALJ810104',
      '6th_aa_PALJ810104',
      '1st_aa_CHOP780206','2nd_aa_CHOP780206','3rd_aa_CHOP780206','4th_aa_CHOP780206','5th_aa_CHOP780206',
      '6th_aa_CHOP780206',
      '1st_aa_ROBB760102','2nd_aa_ROBB760102','3rd_aa_ROBB760102','4th_aa_ROBB760102','5th_aa_ROBB760102',
      '6th_aa_ROBB760102',
       '1st_aa_GEIM800107','2nd_aa_GEIM800107','3rd_aa_GEIM800107','4th_aa_GEIM800107','5th_aa_GEIM800107',
       '6th_aa_GEIM800107',
       '1st_aa_GARJ730101','2nd_aa_GARJ730101','3rd_aa_GARJ730101','4th_aa_GARJ730101','5th_aa_GARJ730101',
       '6th_aa_GARJ730101',
       '1st_aa_FAUJ880110','2nd_aa_FAUJ880110','3rd_aa_FAUJ880110','4th_aa_FAUJ880110','5th_aa_FAUJ880110',
       '6th_aa_FAUJ880110',
       '1st_aa_VENT840101','2nd_aa_VENT840101','3rd_aa_VENT840101','4th_aa_VENT840101','5th_aa_VENT840101',
       '6th_aa_VENT840101',
       '1st_aa_RACS820114','2nd_aa_RACS820114','3rd_aa_RACS820114','4th_aa_RACS820114','5th_aa_RACS820114',
       '6th_aa_RACS820114',
       '1st_aa_ONEK900102','2nd_aa_ONEK900102','3rd_aa_ONEK900102','4th_aa_ONEK900102','5th_aa_ONEK900102',
       '6th_aa_ONEK900102',
       '1st_aa_PTIO830102','2nd_aa_PTIO830102','3rd_aa_PTIO830102','4th_aa_PTIO830102','5th_aa_PTIO830102',
       '6th_aa_PTIO830102',
       '1st_aa_FINA910102','2nd_aa_FINA910102','3rd_aa_FINA910102','4th_aa_FINA910102','5th_aa_FINA910102',
       '6th_aa_FINA910102',
       '1st_aa_MAXF760104','2nd_aa_MAXF760104','3rd_aa_MAXF760104','4th_aa_MAXF760104','5th_aa_MAXF760104',
       '6th_aa_MAXF760104',
       '1st_aa_ZIMJ680103','2nd_aa_ZIMJ680103','3rd_aa_ZIMJ680103','4th_aa_ZIMJ680103','5th_aa_ZIMJ680103',
       '6th_aa_ZIMJ680103',
       '1st_aa_QIAN880123','2nd_aa_QIAN880123','3rd_aa_QIAN880123','4th_aa_QIAN880123','5th_aa_QIAN880123',
       '6th_aa_QIAN880123',
       '1st_aa_AURR980106','2nd_aa_AURR980106','3rd_aa_AURR980106','4th_aa_AURR980106','5th_aa_AURR980106',
       '6th_aa_AURR980106',
       '1st_aa_FINA910102','2nd_aa_FINA910102','3rd_aa_FINA910102','4th_aa_FINA910102','5th_aa_FINA910102',
    '6th_aa_FINA910102',
        #'1st_aa_DAWD720101','2nd_aa_DAWD720101','3rd_aa_DAWD720101','4th_aa_DAWD720101','5th_aa_DAWD720101',
        #'6th_aa_DAWD720101',
        #'1st_aa_BIGC670101','2nd_aa_BIGC670101','3rd_aa_BIGC670101','4th_aa_BIGC670101','5th_aa_BIGC670101',
        #'6th_aa_BIGC670101',
        '1st_aa_KLEP840101','2nd_aa_KLEP840101','3rd_aa_KLEP840101','4th_aa_KLEP840101','5th_aa_KLEP840101',
        '6th_aa_KLEP840101',
        
]

feat.extend(x.columns[0:120])
x = x.loc[:,feat]
print(x)

In [ ]:

## machine learning models ##

from sklearn.cross_validation import train_test_split
data_train, data_test, target_train, target_test = train_test_split(x, y)
len(data_train)

from sklearn import cross_validation
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
estimators = {}
estimators['ranfom_forest'] = RandomForestClassifier(n_estimators = 1000)
estimators['svm_c_rbf'] = svm.SVC(C=0.01)
estimators['svm_c_linear'] = svm.SVC(kernel='linear',C=0.01)
estimators['logistic'] = LogisticRegression()


In [ ]:
for k in estimators.keys():
    print('----%s----' % k)
    estimators[k] = estimators[k].fit(data_train,target_train)
    pred = estimators[k].predict(data_test)
    scores = cross_validation.cross_val_score(estimators[k], data_test, target_test, cv=10)
    print("%s Cross Avg. Score: %0.2f (+/- %0.2f)" % (k, scores.mean(), scores.std()))

